In [2]:
import datetime
import os
import pandas as pd
import numpy as np
from pprint import pprint
import requests
# Scrappings tools for downloading
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import re

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
qualifiying_weather = pd.read_csv('../../../scrapping/data/qualifiying_weather.csv')

In [3]:
qualifiying_weather.loc[qualifiying_weather["weather"] == '', :]

,gp_title,weather,year,title,url


In [4]:
qualifiying_weather["weather"].str.split("\n").apply(lambda x : x[0])

0                 Weather: dry  18°C
1                 Weather: dry  26°C
2             Weather: wet/dry  21°C
3                 Weather: dry  16°C
4                  Weather: dry 27°C
                   ...              
154       Weather: Dry, clouded 15ºC
155       Weather: Dry, clouded 27ºC
156    Weather: Dry, clowdy sky 28ºC
157                     Weather: Dry
158                     Weather: Dry
Name: weather, Length: 159, dtype: object

In [5]:
t = 'Weather: dry  18°C\nTarmac: dry  25°C\nHumidity: 86%\nWind: 1 km/h SW\nPressure: 962 mbar'

In [6]:
t.split("\n")[0].split(":")[1].split(" ")[1]

'dry'

In [7]:
qualifiying_weather['clean_weather'] = qualifiying_weather["weather"].str.split("\n")\
                                            .apply(lambda x : x[0]).str.split(":")\
                                            .apply(lambda x : re.sub('[°C0-9]', '', x[1])
                                                   .replace("º","").replace('.-.','')
                                                   .replace(' ','').replace('--','')
                                                   .replace('-','').replace(' - ','')
                                                   .replace('.,','').replace(',.','')
                                                   .replace(' .', '').replace(' ,', ''))

In [8]:
qualifiying_weather['clean_weather'] = qualifiying_weather['clean_weather']\
            .apply(lambda x: x[:-1] if x.endswith(',') or x.endswith('.') else x)

In [9]:
qualifiying_weather['clean_weather'] = qualifiying_weather['clean_weather']\
            .apply(lambda x: x[1:] if x.startswith('.') or x.startswith(',') or x.startswith(' .') else x)

In [10]:
qualifiying_weather['clean_weather'] = qualifiying_weather["clean_weather"].str.replace('/',',')

In [11]:
qualifiying_weather['clean_weather'] = qualifiying_weather["clean_weather"].str.replace('&',',')

In [12]:
qualifiying_weather['clean_weather'] = qualifiying_weather['clean_weather'].str.lower()

In [13]:
qualifiying_weather['year'] = qualifiying_weather['year'].astype(str) 

In [14]:
qualifiying_weather['year'] = qualifiying_weather['year'].str.split(".").apply(lambda x : x[0])

In [15]:
qualifiying_weather.drop(118, inplace=True)

In [16]:
cleaned_df = qualifiying_weather[['gp_title', 'year', 'clean_weather']]

In [17]:
cleaned_df

,gp_title,year,clean_weather
0,Belgian Grand Prix,2023,dry
1,Hungarian F1 Grand Prix,2023,dry
2,British F1 Grand Prix,2023,"wet,dry"
3,Austrian F1 Grand Prix,2023,dry
4,Austrian F1 GP,2023,dry
...,...,...,...
154,Russian Grand Prix,2015,"dry,clouded"
155,Japanese Grand Prix,2015,"dry,clouded"
156,Singapore Grand Prix,2015,"dry,clowdysky"
157,US F1 Grand Prix,2012,dry


In [18]:
if not os.path.exists('./output'):
    os.mkdir('./output')
cleaned_df.to_csv('./stagging/stagging_weather.csv', index = False)